In [53]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])

        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [52]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation



V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])
@time frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)


  0.000993 seconds (4.89 k allocations: 306.305 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1, 1]  =  -1
  [4, 1]  =  -1
  [1, 2]  =  1
  [2, 2]  =  -1
  [2, 3]  =  1
  [3, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  1
  [1, 3]  =  1
  [1, 4]  =  -1)

In [51]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  621.000 μs (4891 allocations: 306.30 KiB)


([0.0 0.0 1.0; 0.0 1.0 1.0; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1, 1]  =  -1
  [4, 1]  =  -1
  [1, 2]  =  1
  [2, 2]  =  -1
  [2, 3]  =  1
  [3, 3]  =  -1
  [3, 4]  =  1
  [4, 4]  =  1, 
  [1, 1]  =  1
  [1, 2]  =  1
  [1, 3]  =  1
  [1, 4]  =  -1)

In [85]:
function duetrasf(V,EV,FE,sigma,vs_num) 
  
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    
    return tV, sV, sEV, M, sigmavs

    end

duetrasf (generic function with 1 method)

In [86]:
function sigma(tV,EV,FE,sV,sEV,sp_idx,sigma)
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])

        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    
    end
    return tmpV,tmpEV,sV,sEV
    
end
        
        
    


sigma (generic function with 2 methods)

In [87]:
function arrsigma(V,EF,FE,sp_idx,sigma)
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return sV,nV,nEV,nFE,nsize
end

arrsigma (generic function with 1 method)

In [100]:
function frag_faceOTT(V, EV, FE, sp_idx, sigma)
    vs_num = size(V, 1)
    tV,sV,sEV,M,sigmavs = duetrasf(V,EV,FE,sigma,vs_num)
    tmpV,tmpEV,sV,sEV = sigma(tV,EV,FE,sV,sEV,sp_idx,sigma)
    sV,nV,nEV,nFE,nsize= arrsigma(sV,sEV,sigmavs,nsize,nV)
    
    return nV, nEV, nFE

end


frag_faceOTT (generic function with 1 method)

In [101]:
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation



V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])
@time frag_faceOTT(Lar.Points(V),EV,FE,[2,4,5],3)

LoadError: MethodError: objects of type Int64 are not callable